# Exercício 4

Dada uma barra com uma carga $P$ aplicada no meio do seu vão, conforme indicado na Figura, pede-se, utilizando elementos finitos lineares, calcular os deslocamentos da mesma adotando-se uma divisão de 2, 4 e 6 elementos lineares. Comparar os campos de deslocamentos obtidos com os deslocamentos exatos para o problema em questão. Considere:

* $EA = 10^5 \cdot (N + 1)$ (Newton) 
* $P = (N + 1) \cdot 10^3$ (Newton)
* $l = 2 + \frac{N+1}{20}$ (metro)



#### Resolução



### Substituindo $N$

Substituindo $N = 0$, temos:

* $EA = 10^5$ (Newton)
* $P = 10^3$ (Newton)
* $l = \frac{41}{20}$ (metro)

### Equação do problema

$$
-EA \frac{\partial ^2 u(x)}{\partial x^2} = P \delta \left( x-\frac{L}{2} \right), 0 \leq x \leq L
$$

onde $\delta$ é a função delta de Dirac.

### Condições de Contorno

Da figura, temos que $u(0) = 0$ (condição de Dirichlet).

### Aproximação Linear

Neste problema utilizaremos somente a aproximação linear, na forma:

$$
u = u_1 \phi_1 + u_2 \phi_2
$$

## Aproximação com 2 Elementos



### Definindo variáveis e calculando $h$

In [2]:
import sympy as sp

E, A, P, x, h, K, L = sp.symbols('E A P x h K L')

n_elements = 2
h = L/n_elements

### Matriz de Rigidez por elemento

Inicialmente, calculamos a matriz de rigidez conforme calculado no exercício anterior:

In [3]:
phi_1 = 1 - x/h
phi_2 = x/h

K = E * A * sp.Matrix([
    [sp.integrate(sp.diff(phi_1, x) * sp.diff(phi_1, x), (x, 0, h)), sp.integrate(sp.diff(phi_1, x) * sp.diff(phi_2, x), (x, 0, h))], 
    [sp.integrate(sp.diff(phi_2, x) * sp.diff(phi_1, x), (x, 0, h)), sp.integrate(sp.diff(phi_2, x) * sp.diff(phi_2, x), (x, 0, h))]
])
display(K)

Matrix([
[ 2*A*E/L, -2*A*E/L],
[-2*A*E/L,  2*A*E/L]])

### Matriz de Rigidez Global

Diferentemente da aproximação linear abordada anteriormente, neste caso como temos mais do que apenas 1 elemento, será necessário calcular a Matriz de Rigidez Global conforme:

$$
K =
\begin{bmatrix}
k_{11}^1    & k_{12}^1              & 0                     & 0 & \cdots & 0 & 0 \\ 
k_{21}^1    & k_{22}^1 + k_{11}^2   & k_{12}^2              & 0 & \cdots & 0 & 0 \\ 
0           & k_{21}^2              & k_{12}^2 + k_{11}^3   & k_{12}^3 & \cdots & 0 & 0 \\ 
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\ 
0 & 0 & 0 & 0 & \cdots & k_{21}^e & k_{22}^e
\end{bmatrix}
$$

onde $e$ é o número de elementos.


In [4]:
K_G = sp.zeros(n_elements + 1)

for i in range(n_elements):
    K_G[i:i+2, i:i+2] += K

display(K_G)

Matrix([
[ 2*A*E/L, -2*A*E/L,        0],
[-2*A*E/L,  4*A*E/L, -2*A*E/L],
[       0, -2*A*E/L,  2*A*E/L]])

### Vetor Fonte Global

In [7]:
def dirac(d):
    if d == L/2:
        return 1
    return 0

F = sp.zeros(n_elements + 1, 1)

for i in range(n_elements):
    F[i, 0] = P * dirac(i*h)

display(F)

Matrix([
[0],
[P],
[0]])

### Aplicando Condições de Contorno

